In [1]:
#Imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings

import seaborn as sns
import matplotlib.patches as mpatches

from sklearn.metrics import ConfusionMatrixDisplay
from sklearn import metrics
from sklearn.metrics import f1_score
from sklearn.model_selection import GridSearchCV
from sklearn import svm



In [2]:
def conf_matrix_classifier_from_predictions(classifier, X, y, name = ""):
    disp = ConfusionMatrixDisplay.from_estimator(
        classifier,
        X,
        y,
        display_labels=['0','1'],
        cmap=plt.cm.Blues,
        normalize=None,
    )
    disp.ax_.set_title('Confusion Matrix: ' + name)
    plt.show()

## Load data

In [3]:
df_encoded = pd.read_csv("data/encoded_data.csv", sep="\t")
df = pd.read_csv("data/combined_dataframe_clean.csv", sep="\t")

## Split dataframe

In [4]:
train_years= [2019, 2020, 2021]
test_years = [2022]

y_train_years_GP = df[df['Year'].isin(train_years)]['GoodPitStop']
y_test_years_GP= df[df['Year'].isin(test_years)]['GoodPitStop']
X_train_years_GP = df_encoded.loc[y_train_years_GP.index]
X_test_years_GP = df_encoded.loc[y_test_years_GP.index]

y_train_years_HP = df[df['Year'].isin(train_years)]['HasPitLap']
y_test_years_HP= df[df['Year'].isin(test_years)]['HasPitLap']
X_train_years_HP = df_encoded.loc[y_train_years_HP.index]
X_test_years_HP = df_encoded.loc[y_test_years_HP.index]


from sklearn.model_selection import train_test_split 
X_train_GP, X_test_GP, y_train_GP, y_test_GP = train_test_split(df_encoded, df['GoodPitStop'], test_size=0.3, random_state=4815)
X_train_HP, X_test_HP, y_train_HP, y_test_HP = train_test_split(df_encoded, df['HasPitLap'], test_size=0.3, random_state=4815)

## PCA

In [5]:
from sklearn.decomposition import PCA

# Create a PCA object with the specified variance threshold
pca = PCA(n_components=0.95)

# Fit the PCA on the training data and transform both the training and test data
X_train_pca = pca.fit_transform(X_train_HP)
X_test_pca = pca.transform(X_test_HP)

# Print the number of features after PCA
print(f"Number of features after PCA: {X_train_pca.shape[1]}")

Number of features after PCA: 48


## SVM

In [6]:

#Creation an SVM classifier
svm_model = svm.SVC(class_weight="balanced")

#Definition the parameter grid for hyperparameter tuning
param_grid = {
    'C': [0.1, 1, 10],
    'kernel': ['linear', 'rbf', 'poly'],
    'gamma': [0.1, 1, 10]
}

# Perform cross-validation and grid search
grid_search = GridSearchCV(estimator=svm_model, param_grid=param_grid, scoring='f1', cv=5)
grid_search.fit(X_train_HP, y_train_HP)

GridSearchCV(cv=5, estimator=SVC(class_weight='balanced'),
             param_grid={'C': [0.1, 1, 10], 'gamma': [0.1, 1, 10],
                         'kernel': ['linear', 'rbf', 'poly']},
             scoring='f1')

In [8]:
#best hyperparameters and evaluate on the validation set
best_svm = grid_search.best_estimator_
y_test_pred = best_svm.predict(X_test_HP)
f1_test = f1_score(y_test_HP, y_test_pred)
print("Best hyperparameters:", grid_search.best_params_)
print("Test F1 score:", f1_test)

Best hyperparameters: {'C': 0.1, 'gamma': 1, 'kernel': 'poly'}
Test F1 score: 0.6209598432908913


In [9]:
print("Grid search results:")
for params, mean_score, std_score in zip(grid_search.cv_results_['params'],
                                         grid_search.cv_results_['mean_test_score'],
                                         grid_search.cv_results_['std_test_score']):
    print("Hyperparameters:", params)
    print("Mean F1 score:", mean_score)
    print("Standard deviation:", std_score)
    print()

Grid search results:
Hyperparameters: {'C': 0.1, 'gamma': 0.1, 'kernel': 'linear'}
Mean F1 score: 0.2237051502035942
Standard deviation: 0.004541904465546698

Hyperparameters: {'C': 0.1, 'gamma': 0.1, 'kernel': 'rbf'}
Mean F1 score: 0.3025727070478638
Standard deviation: 0.0066911232443225156

Hyperparameters: {'C': 0.1, 'gamma': 0.1, 'kernel': 'poly'}
Mean F1 score: 0.3394181200817622
Standard deviation: 0.010219004220740503

Hyperparameters: {'C': 0.1, 'gamma': 1, 'kernel': 'linear'}
Mean F1 score: 0.2237051502035942
Standard deviation: 0.004541904465546698

Hyperparameters: {'C': 0.1, 'gamma': 1, 'kernel': 'rbf'}
Mean F1 score: 0.0
Standard deviation: 0.0

Hyperparameters: {'C': 0.1, 'gamma': 1, 'kernel': 'poly'}
Mean F1 score: 0.5592521428296783
Standard deviation: 0.032763234812199504

Hyperparameters: {'C': 0.1, 'gamma': 10, 'kernel': 'linear'}
Mean F1 score: 0.2237051502035942
Standard deviation: 0.004541904465546698

Hyperparameters: {'C': 0.1, 'gamma': 10, 'kernel': 'rbf'}
Mea

### With PCA

In [6]:
svm_model = svm.SVC(class_weight="balanced")

param_grid = {
    'C': [0.1, 1, 10],
    'kernel': ['linear', 'rbf', 'poly'],
    'gamma': [0.1, 1, 10]
}

grid_search_pca = GridSearchCV(estimator=svm_model, param_grid=param_grid, scoring='f1', cv=5)
grid_search_pca.fit(X_train_pca, y_train_HP)

In [ ]:
#best hyperparameters 
best_svm = grid_search_pca.best_estimator_
y_test_pred = best_svm.predict(X_test_HP)
f1_test = f1_score(y_test_HP, y_test_pred)
print("Best hyperparameters:", grid_search_pca.best_params_)
print("Test F1 score:", f1_test)

AttributeError: 'GridSearchCV' object has no attribute 'best_estimator_'

In [ ]:
### With good pit stop

In [ ]:
svm_model = svm.SVC(class_weight="balanced")

param_grid = {
    'C': [0.1, 1, 10],
    'kernel': ['linear', 'rbf', 'poly'],
    'gamma': [0.1, 1, 10]
}

grid_search_gp = GridSearchCV(estimator=svm_model, param_grid=param_grid, scoring='f1', cv=5)
grid_search_gp.fit(X_train_GP, y_train_GP)

In [ ]:
best_svm = grid_search_gp.best_estimator_
y_test_pred = best_svm.predict(X_test_HP)
f1_test = f1_score(y_test_HP, y_test_pred)
print("Best hyperparameters:", grid_search_gp.best_params_)
print("Test F1 score:", f1_test)

In [ ]:
### With year split

In [ ]:
#Definition the parameter grid for hyperparameter tuning
param_grid = {
    'C': [0.1, 1, 10],
    'kernel': ['linear', 'rbf', 'poly'],
    'gamma': [0.1, 1, 10]
}

# Perform cross-validation and grid search
grid_search_years = GridSearchCV(estimator=svm_model, param_grid=param_grid, scoring='f1', cv=5)
grid_search_years.fit(X_train_years_HP, y_train_years_HP)

In [ ]:
#best hyperparameters and evaluate on the validation set
best_svm = grid_search_years.best_estimator_
y_test_pred = best_svm.predict(X_test_HP)
f1_test = f1_score(y_test_HP, y_test_pred)
print("Best hyperparameters:", grid_search_years.best_params_)
print("Test F1 score:", f1_test)

## Random forest